In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.8 MB/s eta 0:00:00


In [3]:
from datasets import Dataset
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, pipeline
import torch
from transformers import TrainingArguments, Trainer

In [4]:
torch.cuda.empty_cache()

In [5]:
df = pd.read_pickle('/content/drive/My Drive/dataset/yelp_2013_balanced.pkl')

In [6]:
df = df[['text','stars']]
df = df.reset_index(drop=True)

In [7]:
df

,text,stars
0,It's a shame that my family has been frequenti...,1
1,My general opinion of writing Yelp reviews is ...,1
2,made reservations on thursday night for saturd...,1
3,"It was a night I hope to forget, but I'll stil...",1
4,Trainwreck is appropriately named based on tod...,1
...,...,...
12495,Had a wonderful brunch with girlfriends today!...,5
12496,"Newer is always better, but not in this case. ...",5
12497,I love this place! The prices are reasonable a...,5
12498,"I ordered a whole tomato pie, a whole chicken ...",5


In [8]:
df = df.reset_index(drop=True)

In [9]:
df['stars'] = df['stars'] - 1

In [10]:
df

,text,stars
0,It's a shame that my family has been frequenti...,0
1,My general opinion of writing Yelp reviews is ...,0
2,made reservations on thursday night for saturd...,0
3,"It was a night I hope to forget, but I'll stil...",0
4,Trainwreck is appropriately named based on tod...,0
...,...,...
12495,Had a wonderful brunch with girlfriends today!...,4
12496,"Newer is always better, but not in this case. ...",4
12497,I love this place! The prices are reasonable a...,4
12498,"I ordered a whole tomato pie, a whole chicken ...",4


In [11]:
dataX = df['text']
dataY = df['stars']
dataX.head(5), dataY.head(5)

(0    It's a shame that my family has been frequenti...
 1    My general opinion of writing Yelp reviews is ...
 2    made reservations on thursday night for saturd...
 3    It was a night I hope to forget, but I'll stil...
 4    Trainwreck is appropriately named based on tod...
 Name: text, dtype: object,
 0    0
 1    0
 2    0
 3    0
 4    0
 Name: stars, dtype: int64)

In [12]:
train_ratio = 0.80
validation_ratio = 0.10
test_ratio = 0.10

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(dataX, dataY, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

In [13]:
raw_train_df = pd.concat([x_train, y_train], axis=1)
raw_test_df = pd.concat([x_test, y_test], axis=1)
raw_val_df = pd.concat([x_val, y_val], axis=1)

In [14]:
raw_train_ds = Dataset.from_pandas(raw_train_df)
raw_test_ds = Dataset.from_pandas(raw_test_df)
raw_val_ds = Dataset.from_pandas(raw_val_df)

In [15]:
print(raw_train_ds, raw_val_ds, raw_test_ds)

Dataset({
    features: ['text', 'stars', '__index_level_0__'],
    num_rows: 10000
}) Dataset({
    features: ['text', 'stars', '__index_level_0__'],
    num_rows: 1250
}) Dataset({
    features: ['text', 'stars', '__index_level_0__'],
    num_rows: 1250
})


In [16]:
BASE_MODEL = "bert-base-uncased"
LEARNING_RATE = 2e-5
MAX_LENGTH = 512
BATCH_SIZE = 16
EPOCHS = 5

id2label = {k:k for k in range(5)}
label2id = {k:k for k in range(5)}

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:
ds = {"train": raw_train_ds, "validation": raw_val_ds, "test": raw_test_ds}

def preprocess_function(examples):
    label = examples["stars"]
    examples = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
    examples["label"] = label
    return examples

for split in ds:
    ds[split] = ds[split].map(preprocess_function, remove_columns=["text", "stars"])

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="../models/yelp-2013-bert",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    weight_decay=0.01,
)

In [ ]:
from transformers import Trainer

torch.manual_seed(42)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.983801,0.596000
2,No log,0.940782,0.652000
3,No log,0.948092,0.694000
4,0.874800,0.943488,0.738000
5,0.874800,0.982227,0.723200


TrainOutput(global_step=625, training_loss=0.7829299499511718, metrics={'train_runtime': 260.3295, 'train_samples_per_second': 38.413, 'train_steps_per_second': 2.401, 'total_flos': 2631181424640000.0, 'train_loss': 0.7829299499511718, 'epoch': 5.0})

In [ ]:
trainer.eval_dataset=ds["test"]
trainer.evaluate()

{'eval_loss': 0.8855817794799805,
 'eval_accuracy': 0.687,
 'eval_runtime': 2.0639,
 'eval_samples_per_second': 121.13,
 'eval_steps_per_second': 7.752,
 'epoch': 5.0}

In [ ]:
input_texts = ["This restaurant is amazing and has the best soup",
               "This restaurant is just okay, can be better. But the drinks are good",
               "Best sushi I've ever had in Tokyo",
               "The drinks are decent. I really like the dumplings",
               "The price is too expensive and the food quality is bad. The waiters were really nice to me, though."
              ]

# Encode the text
encoded = tokenizer(input_texts, truncation=True, padding="max_length", max_length=512, return_tensors="pt").to("cuda")

# Call the model to predict under the format of logits of 5 classes
logits = model(**encoded).logits

# Get the class
torch.argmax(logits, axis=1)

tensor([4, 2, 4, 3, 1], device='cuda:0')

In [ ]:
#trainer.save_model('./drive/MyDrive/yelp-longformer-500')

In [ ]:
#tokenizer.save_pretrained("./drive/MyDrive/yelp-longformer-tokenizer-500")